# Gradient Descent Implementation Challenge!!

## Use gradient descent to find the optimal parameters of a **multiple** regression model. (We only showed an implementation for a bivariate model during lecture.)

A note: Implementing gradient descent in any context is not trivial, particularly the step where we calculate the gradient will change based on the number of parameters that we're trying to optimize for. You will need to research what the gradient of a multiple regression model looks like. This challenge is pretty open-ended but I hope it will be thrilling. Please work together, help each other, share resources and generally expand your understanding of gradient descent as you try and achieve this implementation. 

## Suggestions:

Start off with a model that has just two $X$ variables You can use any datasets that have at least two x variables. Potential candidates might be the blood pressure dataset that we used during lecture on Monday: [HERE](https://college.cengage.com/mathematics/brase/understandable_statistics/7e/students/datasets/mlr/excel/mlr02.xls) or any of the housing datasets. You would just need to select from them the two varaibles $x$ variables and one y variable that you want to work with that you most want to work with. 

Use Sklearn to find the optimal parameters of your model first. (like we did during the lecture.) So that you can compare the parameter estimates of your gradient-descent linear regression to the estimates of OLS linear regression. If implemented correctly they should be nearly identical.

Becoming a Data Scientist is all about striking out into the unknown, getting stuck and then researching and fighting and learning until you get yourself unstuck. Work together! And fight to take your own learning-rate fueled step towards your own optimal understanding of gradient descent! 


In [18]:
##### Make it Hap'n Cap'n #####
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures
from functools import reduce
from sklearn.preprocessing import StandardScaler
from math import sqrt


url = 'https://raw.githubusercontent.com/ryanleeallred/datasets/master/Ames%20Housing%20Data/train.csv'

data = pd.read_csv(url)

print(data.shape)
data.head()

(1460, 81)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [19]:
df = data.fillna(method='ffill')

cols = ('MSSubClass','MSZoning','Street','Alley','LotShape','LandContour','Utilities','LotConfig',
        'LandSlope','Neighborhood', 'Condition1','Condition2','BldgType','HouseStyle','RoofStyle',
        'RoofMatl','Exterior1st','Exterior2nd','MasVnrType','ExterQual','ExterCond','Foundation',
        'BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','Heating','HeatingQC',
        'CentralAir','Electrical','KitchenQual','Functional','FireplaceQu','GarageType','GarageFinish',
        'GarageQual','GarageCond','PavedDrive','PoolQC','Fence','MiscFeature','SaleType','SaleCondition') 

for i in cols:
    
    LE = LabelEncoder() 
    LE.fit(list(df[i].values)) 
    df[i] = LE.transform(list(df[i].values))

In [43]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,5,3,65.0,8450,1,2,3,3,0,...,0,3,4,4,0,2,2008,8,4,208500
1,2,0,3,80.0,9600,1,2,3,3,0,...,0,3,4,4,0,5,2007,8,4,181500
2,3,5,3,68.0,11250,1,2,0,3,0,...,0,3,4,4,0,9,2008,8,4,223500
3,4,6,3,60.0,9550,1,2,0,3,0,...,0,3,4,4,0,2,2006,8,0,140000
4,5,5,3,84.0,14260,1,2,0,3,0,...,0,3,4,4,0,12,2008,8,4,250000


In [21]:
df_norm = pd.DataFrame(StandardScaler().fit_transform(df),columns=df.columns)

df_norm['ones'] = np.ones(df.shape[0])
df_norm.head()

/Users/chrislouie/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/chrislouie/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,ones
0,-1.730865,0.200350,-0.045532,-0.214138,-0.207142,0.064238,2.865127,0.750731,0.314667,-0.02618,...,2.022955,3.001304,4.008632,-0.087688,-1.599111,0.138777,0.313867,0.208502,0.347273,1.0
1,-1.728492,-1.001421,-0.045532,0.415088,-0.091886,0.064238,2.865127,0.750731,0.314667,-0.02618,...,2.022955,3.001304,4.008632,-0.087688,-0.489110,-0.614439,0.313867,0.208502,0.007288,1.0
2,-1.726120,0.200350,-0.045532,-0.088293,0.073480,0.064238,2.865127,-1.378933,0.314667,-0.02618,...,2.022955,3.001304,4.008632,-0.087688,0.990891,0.138777,0.313867,0.208502,0.536154,1.0
3,-1.723747,0.440704,-0.045532,-0.423880,-0.096897,0.064238,2.865127,-1.378933,0.314667,-0.02618,...,2.022955,3.001304,4.008632,-0.087688,-1.599111,-1.367655,0.313867,-3.426284,-0.515281,1.0
4,-1.721374,0.200350,-0.045532,0.582881,0.375148,0.064238,2.865127,-1.378933,0.314667,-0.02618,...,2.022955,3.001304,4.008632,-0.087688,2.100892,0.138777,0.313867,0.208502,0.869843,1.0


In [68]:
X = df_norm.loc[:,'MoSold':'YrSold']
ones = np.ones([X.shape[0],1])
X = np.concatenate((ones,X),axis=1)

y = df_norm.loc[:,['SalePrice']].values
theta = np.zeros([1,3])
alpha = 0.01
iters = 1000
n = y.size

print(X.shape)
print(y.shape)
print(X)

(1460, 3)
(1460, 1)
[[ 1.         -1.5991111   0.13877749]
 [ 1.         -0.48911005 -0.61443862]
 [ 1.          0.99089135  0.13877749]
 ...
 [ 1.         -0.48911005  1.64520971]
 [ 1.         -0.8591104   1.64520971]
 [ 1.         -0.1191097   0.13877749]]


In [77]:
model = LinearRegression()
model.fit(X,y)

beta_1 = model.coef_[0]
beta_0 = model.intercept_

print(beta_1)
print(beta_0)

#i really don't think this is right

[ 0.          0.04313353 -0.02263711]
[9.40578872e-16]


In [70]:
# def calc_cost(theta,X,y,n):
#     '''
#     Calculate the cost for given X and y for a single dimension of X
#     theta = vector of thetas
#     X = Row X's np.zeros
#     y = y's np.zeros
#     '''
    
#     preds = np.dot(X,theta)
    
#     cost = (1/2*n) * np.sum(np.square(preds-y))
#     return cost

In [71]:
def computeCost(X,y,theta):
    tobesummed = np.power(((X @ theta.T)-y),2)
    return np.sum(tobesummed)/(2 * len(X))

In [72]:
# def gradient_descent(X,y,theta,iterations,alpha,n):
    
#     cost = np.zeros(iterations)
#     preds = np.dot(X,theta)
#     error = preds - y
    
#     for i in range(iterations):

#         theta = theta - (alpha * 1/n) * np.dot(X.T,error)
#         cost[i] = calc_cost(X,y,theta,n)
    
#     return theta,cost

In [73]:
def gradientDescent(X,y,theta,iters,alpha):
    cost = np.zeros(iters)
    for i in range(iters):
        theta = theta - (alpha/len(X)) * np.sum(X * (X @ theta.T - y), axis=0)
        cost[i] = computeCost(X, y, theta)
    
    return theta,cost


In [74]:
gradientDescent(X,y,theta,iters,alpha)

(array([[ 9.34509699e-16,  4.31312829e-02, -2.26387061e-02]]),
 array([0.49997024, 0.49994116, 0.49991273, 0.49988493, 0.49985776,
        0.49983121, 0.49980525, 0.49977987, 0.49975506, 0.49973081,
        0.49970711, 0.49968394, 0.49966129, 0.49963914, 0.4996175 ,
        0.49959634, 0.49957566, 0.49955544, 0.49953568, 0.49951635,
        0.49949747, 0.499479  , 0.49946096, 0.49944331, 0.49942607,
        0.49940921, 0.49939272, 0.49937661, 0.49936086, 0.49934547,
        0.49933041, 0.4993157 , 0.49930132, 0.49928726, 0.49927351,
        0.49926007, 0.49924694, 0.49923409, 0.49922154, 0.49920927,
        0.49919727, 0.49918554, 0.49917408, 0.49916287, 0.49915191,
        0.4991412 , 0.49913073, 0.49912049, 0.49911048, 0.4991007 ,
        0.49909113, 0.49908178, 0.49907264, 0.4990637 , 0.49905497,
        0.49904643, 0.49903808, 0.49902991, 0.49902193, 0.49901413,
        0.49900651, 0.49899905, 0.49899176, 0.49898463, 0.49897767,
        0.49897085, 0.49896419, 0.49895768, 0.4989513

## Stretch Goals

If you happen upon the most useful resources for accomplishing this challenge first, I want you to spend time today studying other variations of Gradient Descent-Based Optimizers.

- Try and write a function that can perform gradient descent for arbitarily large (in dimensionality) multiple regression models. 
- Create a notebook for yourself exploring these topics
- How do they differ from the "vanilla" gradient descent we explored today
- How do these different gradient descent-based optimizers seek to overcome the challenge of finding the global minimum among various local minima?
- Write a blog post that reteaches what you have learned about these other gradient descent-based optimizers.

[Overview of GD-based optimizers](http://ruder.io/optimizing-gradient-descent/)

[Siraj Raval - Evolution of Gradient Descent-Based Optimizers](https://youtu.be/nhqo0u1a6fw)